In [22]:
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.animation as animation
#from IPython.display import HTML
#from skimage import color
from scipy.ndimage.filters import convolve
import moviepy.editor as mpy
import matplotlib.image as mpimg
from moviepy.editor import ImageSequenceClip
import cv2
%matplotlib inline

def infection_propagation(SIR_matrice, infection_rate, incubation_rate):
    S,I,R = SIR_matrice
    newly_infected = infection_rate*R*S
    dR = incubation_rate*I
    dS = - newly_infected
    dI = newly_infected - dR
    assembled=np.array([dS, dI, dR])
    return assembled

def disperse_virus(SIR, dispersion_kernel, dispersion_rates):
   
    dispersed=np.zeros((np.shape(SIR)[0], np.shape(SIR)[1], np.shape(SIR)[2]), dtype=float)
    for i in range(0, 3):
        dispersed[i, :, :]=convolve(SIR[i, :, :], dispersion_kernel, cval=0)*dispersion_rates[i]
        
    return dispersed



In [23]:

infection_rate = 0.5
incubation_rate = 0.2

dispersion_kernel = np.array([[0.5, 1 , 0.5],
                                [1  , -6, 1],
                                [0.5, 1, 0.5]]) 
dispersion_rates  = [0, 0.05, 0.03]

population_map = mpimg.imread('swiss_dot_map.png')
swiss_map=population_map[:, :, 2]
resized = cv2.resize(swiss_map, (540, 360), interpolation = cv2.INTER_AREA)
pop_map=(1-resized)

SIR = np.zeros((3, np.shape(resized)[0], np.shape(resized)[1]),  dtype=float)
SIR[0] = pop_map

start = (100, 300) #Patient 0 position

SIR[1,start[0], start[1]] = 0.8 # 

dt = 1.0 
country = {'SIR':SIR, 't':0}
coefs = np.array([2,25,25]).reshape((3,1,1))

ims = []
total_time=600

#Generate list of images for video simmulation
for i in range(total_time): 
    infect = infection_propagation(country['SIR'], infection_rate, incubation_rate)
    disperse = disperse_virus(country['SIR'], dispersion_kernel, dispersion_rates)
    country['SIR'] += dt*(infect+disperse)
    country['t']= country['t']+dt
    accentuated = 255*coefs*country['SIR']
    image = accentuated[::-1].swapaxes(0,2).swapaxes(0,1)  
    frame1=np.minimum(255, image)
    ims.append(frame1)
    



In [24]:
#Write movie
#clip = ImageSequenceClip(list(array), fps=20)
clip = ImageSequenceClip(ims, fps=25)
#clip.write_gif('simulation.gif', fps=20)
clip.write_videofile('simulation.mp4', fps=25)

t:   3%|▎         | 18/600 [00:00<00:03, 176.70it/s, now=None]

Moviepy - Building video simulation.mp4.
Moviepy - Writing video simulation.mp4



Moviepy - Done !
Moviepy - video ready simulation.mp4
